In [1]:
import json
import pandas as pd
from transformers import T5Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
result = []

In [4]:
with open('squad_train.json', "r") as fh:
    source = json.load(fh)
    for article in source["data"]:
        for para in article["paragraphs"]:
            context = para["context"].replace("''", '" ').replace("``", '" ')
            context_tokens = tokenizer.tokenize(context.lower())
            
            for qa in para["qas"]:
                ques = qa["question"].replace("''", '" ').replace("``", '" ')
                ques_tokens = tokenizer.tokenize(ques.lower())

                for answer in qa["answers"]:
                    answer_text = answer["text"]
                    
                    start = answer["answer_start"]
                    end = start + len(answer_text)
                    
                    if len(context_tokens) < 100 and len(ques_tokens) < 30:
                        txt = "context : " + context.lower() + " question : " + ques.lower()
                        ans = answer_text.lower()
                        result.append([txt,ans])
                    else:
                        #s_context = context[:end+150]
                        i = end-1
                        while i < len(context) and context[i] != '.' and context[i] != '?':
                            i = i+1
                        s_context = context[:i+1]
                        
                        s_context_tokens = tokenizer.tokenize(s_context)
                        if len(s_context_tokens) < 100 and len(ques_tokens) < 30:
                            txt = "context : " + s_context.lower() + " question : " + ques.lower()
                            ans = answer_text.lower()
                            result.append([txt,ans])

In [5]:
len(result)

51045

In [6]:
result[40000]

['context : educational psychology can in part be understood through its relationship with other disciplines. it is informed primarily by psychology, bearing a relationship to that discipline analogous to the relationship between medicine and biology. question : how is the method of educational psychology have some relation to medicine and biology?',
 'discipline analogous']

In [7]:
train_df = pd.DataFrame(result , columns = ['Text', 'Answer'])

In [8]:
train_df.to_csv('squad.csv', index=False)